<a href="https://colab.research.google.com/github/Dhaneswar2003/CODSOFT/blob/main/CREDIT_CARD_FRAUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [5]:
pd.set_option('display.max_columns', None)

In [10]:
df = pd.read_csv('/content/sample_data/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [8]:
df.shape

(89220, 31)

In [11]:
df.dtypes

,0
Time,float64
V1,float64
V2,float64
V3,float64
V4,float64
V5,float64
V6,float64
V7,float64
V8,float64
V9,float64


In [12]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [13]:
fraud_count=df["Class"].value_counts()
fraud_rate=100*fraud_count/df.shape[0]
fraud_data=pd.concat([fraud_count, fraud_rate],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,272779,99.823247
1,1.0,482,0.176387


In [15]:
df_fraud=df[df['Class']==1]
df_not_fraud=df[df['Class']==0]
df_not_fraud_sampled=df_not_fraud.sample(df_fraud.shape[0], replace=False, random_state=101)
df_balanced=pd.concat([df_not_fraud_sampled, df_fraud], axis=0).sample(frac=1, replace=False, random_state=101).reset_index().drop('index', axis=1)
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,77249.0,-0.918301,0.471019,1.996879,-1.059160,-0.702247,0.060891,0.186768,-0.039287,1.096876,-1.172577,-0.093008,1.120369,1.313132,-0.588907,1.605285,-1.201646,0.292967,-0.021079,0.978802,-0.006925,0.289942,1.136383,-0.230720,0.168195,0.111844,-0.461433,-0.167489,0.148582,50.00,0.0
1,34684.0,-2.439237,2.591458,-2.840126,1.286244,-1.777016,-1.436139,-2.206056,-2.282725,-0.292885,-3.717450,3.907399,-7.220004,-1.211739,-9.657627,0.927518,-4.738662,-9.276636,-3.081961,0.177746,0.513530,1.774460,-0.771390,0.065727,0.103916,-0.057578,0.242652,-0.268649,-0.743713,125.30,1.0
2,23663.0,-2.133959,2.712677,-0.330319,0.889011,-0.730310,-0.395682,-0.705281,1.476469,0.283614,-0.851827,-0.635850,-1.674153,3.058507,2.591637,0.151840,0.354496,0.839738,0.462675,0.577009,-0.153290,-0.133550,-0.518370,0.045625,-0.530588,0.024753,-0.507814,-0.288542,-0.002406,9.34,0.0
3,148812.0,-0.333260,-4.238924,-1.279286,1.439904,-2.160105,-0.070001,0.784924,-0.289524,1.419227,-0.726447,-0.933989,0.479621,0.325844,0.040184,1.023274,0.569085,-0.637297,0.483748,-0.648700,2.204352,0.764133,-0.312505,-0.775900,0.020156,-0.822810,-0.557458,-0.169477,0.180189,1197.65,0.0
4,74262.0,-2.250535,2.365755,-2.955491,0.089791,-2.830745,-0.844462,-0.174062,-0.407138,0.174216,-2.998926,2.605169,-3.677717,-0.961255,-4.629481,1.942182,-4.180379,-4.605686,-2.557184,1.589582,-0.486537,0.256560,-0.466245,0.291105,0.242567,-1.279094,-1.123534,-0.630977,0.326839,311.28,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,35585.0,-2.019001,1.491270,0.005222,0.817253,0.973252,-0.639268,-0.974073,-3.146929,-0.003159,-0.121653,0.024370,-1.073812,-2.002769,-0.895434,1.748936,-0.103527,2.031196,0.317598,0.386761,-1.029965,2.839596,-1.185443,-0.142812,-0.086103,-0.329113,0.523601,0.626283,0.152440,0.76,1.0
960,115691.0,-1.550273,1.088689,-2.393388,1.008733,-1.087562,-1.104602,-2.670503,0.147655,-0.978626,-3.514133,3.586042,-4.053358,0.862426,-5.765823,-0.507511,-4.921865,-9.567268,-2.772974,1.717095,0.327490,0.802316,1.037105,0.095815,-0.320191,-0.080731,0.091012,0.024928,-0.077820,17.06,1.0
961,83004.0,-1.821910,0.693055,-0.135168,-0.166781,0.812816,0.022878,-0.357558,0.971444,-0.974522,-0.439805,0.259135,1.203008,1.305511,0.794598,0.175860,0.459852,-0.475560,0.206255,0.295392,-0.083710,0.180638,0.342674,-0.453090,-1.055647,-0.216433,0.376200,0.123336,-0.165313,16.91,0.0
962,17230.0,-0.469327,1.111453,2.041003,1.731595,0.135147,-0.093625,0.266155,0.082988,0.580255,-0.164563,0.500376,-3.021760,0.924213,1.500794,-0.400283,0.706048,0.115114,-0.046796,-2.548522,-0.358709,0.159387,0.592670,-0.053596,0.320748,-0.369121,-0.136605,-0.100845,0.039347,3.93,1.0


In [21]:
fraud_count=df_balanced["Class"].value_counts()
fraud_rate=100*fraud_count/df_balanced.shape[0]
fraud_data=pd.concat([fraud_count, fraud_rate], axis=1).reset_index()
fraud_data.columns=['Class', 'Count','percentage']
fraud_data

,Class,Count,percentage
0,0.0,482,50.0
1,1.0,482,50.0


In [23]:
X_train,X_test,y_train,y_test = train_test_split(df_balanced.drop('Class', axis=1), df_balanced['Class'], test_size=0.2, random_state=101)
print(f'''X_train: {X_train.shape}
X_test: {X_test.shape}
y_train: {y_train.shape}
y_test: {y_test.shape}''')

X_train: (771, 30)
X_test: (193, 30)
y_train: (771,)
y_test: (193,)


In [27]:
randomForestModel =Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])
randomForestModel.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [28]:
y_pred_logis= randomForestModel.predict(X_test)
y_pred_logis

array([0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 0.])

In [29]:
cr=classification_report(y_test, y_pred_logis)
print(cr)

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       102
         1.0       0.99      0.95      0.97        91

    accuracy                           0.97       193
   macro avg       0.97      0.97      0.97       193
weighted avg       0.97      0.97      0.97       193



In [30]:
with open('./model.pkl', 'wb') as fp:
  pickle.dump(randomForestModel, fp)